In [23]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [24]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=False
    debug=False
    print_freq=10
    num_workers=0
    size=224
    model_name='vit_large_patch32_224'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=0
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=32
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed = [42] #[42, 10, 20, 51, 111]
    target_size=1
    target_col='KIc'
    n_fold=5
    kfold="Kfold" #or Kfold
    trn_fold = [i for i in range(n_fold)]
    train=True
    grad_cam=True
      
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './KIc/Model/vit/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [25]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


In [26]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df[CFG.target_col].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        label = torch.tensor(self.labels[idx]).float()
        return image, label

In [27]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            # A.Resize(CFG.size, CFG.size),
            A.RandomResizedCrop(CFG.size, CFG.size, scale=(0.85, 1.0)),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [28]:
# ====================================================
# MODEL
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(self.cfg.model_name, pretrained=pretrained)
        self.n_features = self.model.head.in_features
        self.model.head = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.cfg.target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

In [29]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [30]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    if CFG.apex:#使わないところは消す
        scaler = GradScaler()
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        if CFG.apex:
            with autocast():
                y_preds = model(images)
                loss = criterion(y_preds.view(-1), labels)
        else:
            y_preds = model(images)
            loss = criterion(y_preds.view(-1), labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.6f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds.view(-1), labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [31]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold, seed):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values

    train_dataset = TrainDataset(train_folds, transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, transform=get_transforms(data='train'))

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, pretrained=True)
    model.to(device)

    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    criterion = RMSELoss()

    best_score = np.inf
    best_loss = np.inf

    # for epoch in range(CFG.epochs):
        # eval
    avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
    
    if isinstance(scheduler, ReduceLROnPlateau):
        scheduler.step(avg_val_loss)
    elif isinstance(scheduler, CosineAnnealingLR):
        scheduler.step()
    elif isinstance(scheduler, CosineAnnealingWarmRestarts):
        scheduler.step()

    # scoring
    # score = get_score(valid_labels, preds)

    # if score < best_score:
    #     best_score = score
    #     LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
    torch.save({'model': model.state_dict(), 'preds': preds}, OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_fold{fold}_seed{seed}_best_netweight.pth')
    
    valid_folds['preds'] = torch.load(OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_fold{fold}_seed{seed}_best_netweight.pth', 
                                map_location=torch.device('cpu'))['preds']

    return valid_folds

In [32]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    for seed in CFG.seed:
        LOGGER.info(f"========== seed{seed} ==========")
        seed_torch()

        train = pd.read_csv('/home/yamanaka/Estimate_KIc_with_ViT/Mototake_Analysis/VGG+GP/inout_data.csv', header=None, names=['Id', 'KIc'])
        train['file_path'] = ['/home/yamanaka/Estimate_KIc_with_ViT/Mototake_Analysis/VGG+GP/imagedata/' + str(i) + '.jpg' for i in train['Id']]
        # train = pd.read_csv('../../KIc/Model/vgg/vgg16_Kfold_seed1_oof_df.csv')
        # vit_oof = pd.read_csv('../../KIc/Model/vit/vit_large_patch32_224_Kfold_seed1_oof_df.csv')\
        #     .sort_values(by='Id').reset_index(drop=True)
        # f = open("../../Mototake_Analysis/VGG+GP/Y_pred_rational.pkl","rb")
        # plot_Y_rational = pkl.load(f)
        # f.close()
        # oof['VGG+GP_preds'] = plot_Y_rational[1][3]
        # oof = oof.sort_values(by='Id').reset_index(drop=True)

        # # diff definition
        # oof['vgg_diff'] = abs(oof['preds'] - oof['KIc'])
        # vit_oof['vit_diff'] = abs(vit_oof['preds'] - vit_oof['KIc'])
        # vgg_gp = abs(oof['VGG+GP_preds'] - oof['KIc'])
        # vit_oof['vit_diff_from_vgg'] = oof['vgg_diff'] - vit_oof['vit_diff']
        # vit_oof['vit_absdiff_from_vgg'] = abs(oof['vgg_diff'] - vit_oof['vit_diff'])

        if CFG.debug:
            CFG.epochs = 1
            train = train.sample(n=100, random_state=seed).reset_index(drop=True)

        if CFG.kfold == 'Kfold':
            Fold = KFold(n_splits=CFG.n_fold, shuffle=True, random_state=seed)
            for n, (train_index, val_index) in enumerate(Fold.split(train)):
                train.loc[val_index, 'fold'] = int(n)
            train['fold'] = train['fold'].astype(int)
        elif CFG.kfold == "StratifiedKfold":
            num_bins = int(np.floor(1 + np.log2(len(train))))
            train["bins"] = pd.cut(train[CFG.target_col], bins=num_bins, labels=False)
            Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=seed)
            for n, (train_index, val_index) in enumerate(Fold.split(train, train["bins"])):
                train.loc[val_index, 'fold'] = int(n)
            train['fold'] = train['fold'].astype(int)

        # # train出力 vgg 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            # train.to_csv vggも
            _oof_df = train_loop(train, fold, seed)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)

        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)

        # save result
        oof_df.to_csv(OUTPUT_DIR+f'{CFG.model_name}_{CFG.kfold}_seed{seed}_oof_df_netweight.csv', index=False)

In [33]:
if __name__ == '__main__':
    main()

========== seed42 ==========
========== seed42 ==========
========== seed42 ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========


EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.5301(3.5301) 
EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.7491(3.3912) 


========== fold: 0 result ==========
========== fold: 0 result ==========
========== fold: 0 result ==========
Score: 3.4005
Score: 3.4005
Score: 3.4005
========== fold: 1 training ==========
========== fold: 1 training ==========
========== fold: 1 training ==========


EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.4645(3.4645) 
EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.6869(3.3542) 


========== fold: 1 result ==========
========== fold: 1 result ==========
========== fold: 1 result ==========
Score: 3.3614
Score: 3.3614
Score: 3.3614
========== fold: 2 training ==========
========== fold: 2 training ==========
========== fold: 2 training ==========


...

========== fold: 2 result ==========
========== fold: 2 result ==========
========== fold: 2 result ==========
Score: 3.2740
Score: 3.2740
Score: 3.2740
========== fold: 3 training ==========
========== fold: 3 training ==========
========== fold: 3 training ==========


EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.6398(3.6398) 
EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.7505(3.4796) 


========== fold: 3 result ==========
========== fold: 3 result ==========
========== fold: 3 result ==========
Score: 3.4872
Score: 3.4872
Score: 3.4872
========== fold: 4 training ==========
========== fold: 4 training ==========
========== fold: 4 training ==========


EVAL: [0/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.4890(3.4890) 
EVAL: [2/3] Elapsed 0m 0s (remain 0m 0s) Loss: 3.4515(3.2229) 


========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
Score: 3.2371
Score: 3.2371
Score: 3.2371
========== CV ==========
========== CV ==========
========== CV ==========
Score: 3.3533
Score: 3.3533
Score: 3.3533
